# Статистика по заказам

Представьте что вы работаете Аналитиком Данных в производственной компании, которая занимается отправкой определенных продуктов по миру. У бизнеса есть предположение, что есть определенные страны, с которыми нам не стоит работать, так как доставка в данные страны может быть нецелесообразной.


Ваша задача, провести определенное исследование и предложить в какой стране работать лучше всего а в какой нет. 

In [1]:
import matplotlib.pyplot as plt
import psycopg2 as psy
import pandas as pd

from datetime import datetime as dt

def sql_to_pd(query):
    '''
    Функция стучится в базу данных и обрабатывает указанный в аргументе запрос.
    Затем преобразует полученные данные в датафрейм и возвращает его.
    '''
    try:
        # Connect to an existing database
        connection = psy.connect(user="postgres",
                                  password="Pga1708844!",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")
        # Create a cursor to perform database operations
        cursor = connection.cursor()
        cursor.execute(query)
        colnames = [desc[0] for desc in cursor.description]
        record = cursor.fetchall()
        # Create a dataframe with the data from a database
        data_frame = pd.DataFrame(record, columns=colnames)
        return data_frame
    except Exception as e:
        print("Error while connecting to PostgreSQL", e)
    finally:
        if (connection):
            cursor.close()
            connection.close()

# 1. Покажите страны, с наибольшим количеством активных заказов
Ниже я буду использовать переменную temp как временное хранилище для разных SQL запросов.
Я сделал так, чтобы не нужно было записывать датафреймы в разные переменные и увеличивать их количество.

Поэтому чтобы этот код работал правильно, нужно выполнять его сверху вниз.

In [2]:
# получаем датасет с активными заказами, то есть теми, где shipped_date = null
temp = sql_to_pd("SELECT * FROM orders WHERE shipped_date IS null;")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          21 non-null     int64  
 1   customer_id       21 non-null     object 
 2   employee_id       21 non-null     int64  
 3   order_date        21 non-null     object 
 4   required_date     21 non-null     object 
 5   shipped_date      0 non-null      object 
 6   ship_via          21 non-null     int64  
 7   freight           21 non-null     float64
 8   ship_name         21 non-null     object 
 9   ship_address      21 non-null     object 
 10  ship_city         21 non-null     object 
 11  ship_region       9 non-null      object 
 12  ship_postal_code  21 non-null     object 
 13  ship_country      21 non-null     object 
dtypes: float64(1), int64(3), object(10)
memory usage: 2.4+ KB


In [3]:
# группировка и аггрегирование
countries_orders = temp.groupby('ship_country').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).reset_index()

In [4]:
# финальная подготовка и вывод
countries_max_orders = countries_orders[countries_orders['order_id'] == max(countries_orders['order_id'])].copy()
countries_max_orders.rename(columns={'order_id': 'amount'}, inplace=True)
countries_max_orders

,ship_country,amount
0,USA,3
1,Venezuela,3


# 2. Исследуя уже отправленные заказы, посчитайте количество заказов по странам.

In [5]:
temp = sql_to_pd("SELECT * FROM orders WHERE shipped_date IS NOT null;")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          809 non-null    int64  
 1   customer_id       809 non-null    object 
 2   employee_id       809 non-null    int64  
 3   order_date        809 non-null    object 
 4   required_date     809 non-null    object 
 5   shipped_date      809 non-null    object 
 6   ship_via          809 non-null    int64  
 7   freight           809 non-null    float64
 8   ship_name         809 non-null    object 
 9   ship_address      809 non-null    object 
 10  ship_city         809 non-null    object 
 11  ship_region       314 non-null    object 
 12  ship_postal_code  790 non-null    object 
 13  ship_country      809 non-null    object 
dtypes: float64(1), int64(3), object(10)
memory usage: 88.6+ KB


In [6]:
# группировка и аггрегирование
sent_orders_count = temp.groupby('ship_country').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).reset_index().copy()

In [7]:
# финальная подготовка и вывод
sent_orders_count.rename(columns={'order_id': 'amount'}, inplace=True)
sent_orders_count

,ship_country,amount
0,Germany,120
1,USA,119
2,Brazil,81
3,France,75
4,UK,56
5,Venezuela,43
6,Austria,38
7,Sweden,37
8,Canada,29
9,Mexico,27


# 3. Посчитайте среднее количество продуктов по активным заказам

In [8]:
# Данные о количестве хранятся в таблице order_details
temp = sql_to_pd("SELECT * FROM orders INNER JOIN order_details USING(order_id) WHERE shipped_date IS null;")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          73 non-null     int64  
 1   customer_id       73 non-null     object 
 2   employee_id       73 non-null     int64  
 3   order_date        73 non-null     object 
 4   required_date     73 non-null     object 
 5   shipped_date      0 non-null      object 
 6   ship_via          73 non-null     int64  
 7   freight           73 non-null     float64
 8   ship_name         73 non-null     object 
 9   ship_address      73 non-null     object 
 10  ship_city         73 non-null     object 
 11  ship_region       43 non-null     object 
 12  ship_postal_code  73 non-null     object 
 13  ship_country      73 non-null     object 
 14  product_id        73 non-null     int64  
 15  unit_price        73 non-null     float64
 16  quantity          73 non-null     int64  
 17 

In [9]:
# группировка и аггрегирование
avg_products_in_order = temp.groupby('order_id').agg({'quantity': 'mean'}).sort_values(by='order_id').reset_index()

In [10]:
# финальная подготовка и вывод
avg_products_in_order['quantity'] = avg_products_in_order['quantity'].apply(lambda x: round(x, 2))
avg_products_in_order

,order_id,quantity
0,11008,60.33
1,11019,2.50
2,11039,33.00
3,11040,20.00
4,11045,19.50
5,11051,10.00
6,11054,15.00
7,11058,9.33
8,11059,25.67
9,11061,15.00


# 4. Выведите каждую страну, посчитайте по ней среднее количество продуктов и количество заказов. (При этом учитывайте репрезентативность результата)

In [18]:
temp = sql_to_pd('SELECT * FROM orders INNER JOIN order_details USING(order_id);')
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          2155 non-null   int64  
 1   customer_id       2155 non-null   object 
 2   employee_id       2155 non-null   int64  
 3   order_date        2155 non-null   object 
 4   required_date     2155 non-null   object 
 5   shipped_date      2082 non-null   object 
 6   ship_via          2155 non-null   int64  
 7   freight           2155 non-null   float64
 8   ship_name         2155 non-null   object 
 9   ship_address      2155 non-null   object 
 10  ship_city         2155 non-null   object 
 11  ship_region       856 non-null    object 
 12  ship_postal_code  2100 non-null   object 
 13  ship_country      2155 non-null   object 
 14  product_id        2155 non-null   int64  
 15  unit_price        2155 non-null   float64
 16  quantity          2155 non-null   int64  


In [25]:
# группировка и аггрегирование
country_count = temp\
                .groupby('ship_country')\
                .agg({'quantity': 'mean', 'order_id': 'count'})\
                .sort_values(by=['quantity', 'order_id'], ascending=False)\
                .reset_index()

In [26]:
# финальная подготовка и вывод
country_count.rename(columns={'order_id': 'orders_count', 'quantity': 'products_amount_mean'}, inplace=True)
country_count['products_amount_mean'] = country_count['products_amount_mean'].apply(lambda x: round(x, 2))
country_count

,ship_country,products_amount_mean,orders_count
0,Austria,41.34,125
1,Ireland,30.62,55
2,Germany,28.09,328
3,USA,26.51,352
4,Canada,26.45,75
5,Denmark,25.43,46
6,Venezuela,24.88,118
7,Belgium,24.86,56
8,Switzerland,24.52,52
9,Sweden,23.04,97


# 5. Исследуя уже отправленные заказы, посчитайте сумму выручки по странам.

In [38]:
temp = sql_to_pd("SELECT *\
                    , CASE \
                        WHEN discount = 0 THEN unit_price * quantity\
                        ELSE (unit_price * quantity) - ((unit_price * quantity)* discount) \
                      END AS total_price\
                FROM orders\
                INNER JOIN order_details USING(order_id)\
                WHERE shipped_date IS null;")
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          73 non-null     int64  
 1   customer_id       73 non-null     object 
 2   employee_id       73 non-null     int64  
 3   order_date        73 non-null     object 
 4   required_date     73 non-null     object 
 5   shipped_date      0 non-null      object 
 6   ship_via          73 non-null     int64  
 7   freight           73 non-null     float64
 8   ship_name         73 non-null     object 
 9   ship_address      73 non-null     object 
 10  ship_city         73 non-null     object 
 11  ship_region       43 non-null     object 
 12  ship_postal_code  73 non-null     object 
 13  ship_country      73 non-null     object 
 14  product_id        73 non-null     int64  
 15  unit_price        73 non-null     float64
 16  quantity          73 non-null     int64  
 17 

In [40]:
# группировка и аггрегирование
price_sum = temp\
            .groupby('ship_country')\
            .agg({'total_price': 'sum'})\
            .sort_values(by='total_price', ascending=False)\
            .reset_index()

In [41]:
# финальная подготовка и вывод
price_sum['total_price'] = price_sum['total_price'].apply(lambda x: round(x, 2))
price_sum

,ship_country,total_price
0,Austria,9898.90
1,Brazil,3865.08
2,Venezuela,3763.92
3,Germany,2487.98
4,USA,1965.72
5,Canada,1309.50
6,France,828.75
7,Switzerland,498.10
8,Italy,406.40
9,Argentina,381.00


# 6. Нарисуйте круговые диаграммы по количеству заказов и странам (При этом учитывайте неактивные заказы)

# 7. Нарисуйте круговые диаграммы по выручке и странам. (При этом учитывайте неактивные заказы)овать.

# 8. Нарисуйте круговые диаграммы по количеству заказов и странам (При этом учитывайте активные заказы)

# 9. Нарисуйте круговые диаграммы по выручке и странам. (При этом учитывайте активные заказы)

# 10. Сделайте выводы и предложите в какой стране более выгодно оперировать.